<h2>Installing and Importing Libraries</h2>

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow=='1.15.0'

In [ ]:
# To show current Tensorflow version
# In order to use bert-as-service, according to some online sources, Tensorflow version must be < 2.0
!pip show tensorflow

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: gast, absl-py, six, tensorflow-estimator, opt-einsum, tensorboard, wrapt, termcolor, wheel, astor, protobuf, grpcio, numpy, keras-applications, google-pasta, keras-preprocessing
Required-by: fancyimpute


In [ ]:
!CFLAGS="-Wno-narrowing" pip install textpipe

In [ ]:
# To look at the logs
# Did !pip install textpipe -log LOG_FILE
!cat LOG_FILE

In [ ]:
# Used to pip install bert-as-service
!pip install bert-serving-client

In [ ]:
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 71kB 3.7MB/s 
     |████████████████████████████████| 358kB 14.9MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=314d7166f2f6918641a404d2e9aec6e4dd7d7d25484242cb28bbecfbe97d3165
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for flask-compress: filename=Flask_Compress-1.5.0-cp36-none-any.whl size=5273 sha256=f8cb47140d39e300e8720b9f15dc4a94d93d4d50e4d42660b3498ea5751f8d63
  Stored in directory: /root/.cache/pip/wheels/f7/e9/e4/5afc286be7c87461375e33152558415dfeb0c8f5af3b50e742
Successfully built GPUtil flask-compress


In [ ]:
# Installing PyTorch interface for BERT by HuggingFace
!pip install pytorch-pretrained-bert pytorch-nlp

In [ ]:
!pip install wordninja

     |████████████████████████████████| 542kB 10.3MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp36-none-any.whl size=541552 sha256=985dac4567286afb2bc007f486224933a3337ee0ac65c2f722ac936a3906c076
  Stored in directory: /root/.cache/pip/wheels/22/46/06/9b6d10ed02c85e93c3bb33ac50e2d368b2586248f192a2e22a
Successfully built wordninja


In [ ]:
!pip install pyspellchecker

     |████████████████████████████████| 1.9MB 14.7MB/s 


In [ ]:
!pip install ftfy

     |████████████████████████████████| 71kB 5.4MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=0b399b3710875a07f459b11b0cb6659232e6e171774b7ea86720dbc591f5832b
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ftfy


In [ ]:
import os
import pandas as pd
import numpy as np
import spacy
from textpipe import doc, pipeline
import tensorflow as tf
import tensorflow_hub as hub
# import torch
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
# from sklearn.model_selection import train_test_split
#from pytorch_pretrained_bert import BertTokenizer, BertConfig
#from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
# from tqdm import tqdm, trange
import io
import matplotlib.pyplot as plt
import nltk
from spellchecker import SpellChecker
import string
from ftfy import fix_encoding
from scipy import spatial
import json
import requests
import time
% matplotlib inline
# displays and stores the output of plotting commands in this notebook

In [ ]:
# Specifying GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(torch.cuda.get_device_name(0))
print('Device: {}; Number of GPU: {}'.format(device, n_gpu))

Tesla K80
Device: cuda; Number of GPU: 1


<h2>Loading Data</h2>

In [ ]:
# Upload data from Google Drive (or can just click mount in UI)
from google.colab import drive
drive.mount('/drive')

"""
# To upload files onto Colab from local drive
# Upload downloaded BERT model (as unzipped folder)
from google.colab import files
uploaded = files.upload()
"""

In [ ]:
os.getcwd() # Getting current directory
os.chdir("/content/drive/My Drive/DSTA Internship/articles")

In [ ]:
# Load the data (100 articles)
files = {}
for filename in os.listdir():
    if ".txt" not in filename:
      continue
    if not filename in files:
        with open(filename, "r", encoding="utf-8") as file:
            files[filename] = file.read()
for filename, text in files.items():
    print(filename)

83.txt
38.txt
24.txt
22.txt
89.txt
97.txt
42.txt
29.txt
32.txt
61.txt
91.txt
33.txt
19.txt
50.txt
100.txt
96.txt
99.txt
43.txt
95.txt
18.txt
46.txt
44.txt
52.txt
92.txt
93.txt
94.txt
62.txt
21.txt
98.txt
41.txt
39.txt
78.txt
84.txt
67.txt
56.txt
28.txt
11.txt
34.txt
35.txt
82.txt
47.txt
4.txt
51.txt
3.txt
1.txt
87.txt
66.txt
74.txt
73.txt
20.txt
49.txt
10.txt
86.txt
25.txt
27.txt
60.txt
65.txt
30.txt
17.txt
15.txt
63.txt
72.txt
90.txt
8.txt
64.txt
71.txt
55.txt
69.txt
54.txt
77.txt
88.txt
2.txt
7.txt
16.txt
79.txt
58.txt
9.txt
14.txt
68.txt
70.txt
13.txt
85.txt
48.txt
40.txt
37.txt
80.txt
36.txt
76.txt
53.txt
45.txt
31.txt
57.txt
5.txt
81.txt
75.txt
26.txt
59.txt
12.txt
6.txt
23.txt


In [ ]:
files['83.txt']

'\nSINGAPORE: The People\'s Action Party (PAP) team contesting Tampines GRC will be led by Environment and Water Resources Minister Masagos Zulkifli, with Senior Minister of State for Trade and Industry Koh Poh Koon joining the team and replacing Deputy Prime Minister Heng Swee Keat. Mr Heng will contest in East Coast GRC instead this election.  Dr Koh was a former Member of Parliament for Ang Mo Kio GRC. \n\n\nAdvertisement\n\n\n\n\nAdvertisement\n\n\nThe other three PAP candidates who will be defending their seats are Mr Baey Yam Keng, Mr Desmond Choo and Ms Cheng Li Hui. They will be challenged by a team from the National Solidarity Party (NSP), which is led by party president Reno Fong.  The other members are Mr Mohd Ridzwan Bin Mohammad, Yeo Ren-Yuan, Choong Hon Heng and Vincent Ng. GE2020 live updates: Nomination Day as it happensThe NSP has done battle before in Tampines, losing in 2011 and 2015 to the PAP. \n\nAdvertisement\n\n\n\nAdvertisement\n\n\nThe NSP got 42. 8 per cent o

In [ ]:
# After using textpipe to remove the html tags
doc.Doc(files['83.txt']).clean

'SINGAPORE: The People\'s Action Party (PAP) team contesting Tampines GRC will be led by Environment and Water Resources Minister Masagos Zulkifli, with Senior Minister of State for Trade and Industry Koh Poh Koon joining the team and replacing Deputy Prime Minister Heng Swee Keat. Mr Heng will contest in East Coast GRC instead this election. Dr Koh was a former Member of Parliament for Ang Mo Kio GRC. Advertisement Advertisement The other three PAP candidates who will be defending their seats are Mr Baey Yam Keng, Mr Desmond Choo and Ms Cheng Li Hui. They will be challenged by a team from the National Solidarity Party (NSP), which is led by party president Reno Fong. The other members are Mr Mohd Ridzwan Bin Mohammad, Yeo Ren-Yuan, Choong Hon Heng and Vincent Ng. GE2020 live updates: Nomination Day as it happensThe NSP has done battle before in Tampines, losing in 2011 and 2015 to the PAP. Advertisement Advertisement The NSP got 42. 8 per cent of the votes cast in 2011, against PAP\'s

<h2>Data cleaning: To remove typos (Work in Progress)</h2>

In [ ]:
# Identify words that are not in vocabulary (possibly words that should have a space between them e.g. developmentsDownload)
# Add space between those words. 

def identifyTypos(text):
  words = text.split() # split by whitespace
  # Creates an empty mapping table, with the third argument of the function listing all the chars to remove during the translation process
  table = str.maketrans('', '', string.punctuation) # string.punctuation is a constant provided in Python with a good list of punctuation chars
  # Remove punctuations
  no_punct = [w.translate(table) for w in words] # list of words
  spell = SpellChecker()
  unknown = spell.unknown(no_punct) # unknown contains all words that seem strange
  return unknown

In [ ]:
word_set = set()
for key, value in files.items():
    article = doc.Doc(value).clean
    word_set = word_set|identifyTypos(article)
len(word_set)

1505

In [ ]:
f = open("./words/empty.csv",'w')
for i in word_set:
  f.write(i + "\n")
f.close()
# Now we have a list of strange-looking words, can manually identify words that are not actually typos

In [ ]:
# After identifying the words that are actually typos, use word ninja to split the words and replace these text in the articles.
for i in unknown:
    split = wordninja.split(i)
    """
    if spell.unknown(split) != set():
      continue
    print(split)
    """

<h2>Proceeded without removing typos</h2>

In [ ]:
# Cleaned the text - removed the html tags using textpipe
# Have not removed the typos
# Used fix_encoding in case there's any encoding issue
# Did not remove stopwords / punctuations / pronouns for ease of understanding context in sentence embedding.

dict_art = {}
for key, value in files.items():
    processed = fix_encoding(doc.Doc(value).clean)
    dict_art[key] = processed

In [ ]:
print(dict_art['54.txt'])

SINGAPORE: The Workers' Party (WP) said on Monday night (Jul 6) that it will push for a reconsideration of its proposal for redundancy insurance given the challenges of the post-COVID environment. This and other issues related to employment and workers were discussed in a pre-recorded webcast on Facebook moderated by WP chief Pritam Singh and featuring GE2020 candidates Dennis Tan, Leon Perera, Abdul Shariff Aboo Kassim and Raeesah Khan. Advertisement Advertisement Mr Tan, who is running in Hougang SMC, said that when the party first raised the idea of redundancy insurance in Parliament in 2017, the ruling People's Action Party (PAP) "rejected it outright", but with the COVID crisis, the proposal may be worth re-looking. This is as more businesses are expected to shutter and more workers are facing job losses due to COVID-19 and the ensuing economic fallout. READ: High stakes in GE2020 amid COVID-19 crisis, with biggest challenges lying ahead: PM Lee in virtual rallyGE2020: NSP's Spenc

<h2>Using BERT Embeddings to find Cosine Similarity Score</h2>


In [ ]:
!echo $PWD

/content


In [ ]:
get_ipython().system_raw(
    'bert-serving-start -model_dir /content/drive/My\ Drive/DSTA\ Internship/uncased_L-24_H-1024_A-16/uncased_L-24_H-1024_A-16/ -http_port 3333 &'
)

In [ ]:
# import json
# import requests

def get_embeddings(texts):
    headers = {
        'content-type':'application/json'
    }
    data = {
        "id":324, # change the id if connection refused because of too many requests
        "texts":texts,
        "is_tokenized": False
    }
    data = json.dumps(data)
    r = requests.post("http://localhost:3333/encode", data=data, headers=headers).json()
    return r['result']

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
dict_emb = {}
for key, value in dict_art.items():
  sentences = nltk.tokenize.sent_tokenize(value)
  embeddings = np.array(get_embeddings(sentences))
  # using mean sentence embeddings to calc 'document embedding'
  dict_emb[key] = np.average(embeddings, axis = 0) # Returns an average array

In [ ]:
dict_emb["5.txt"]

array([-0.18737789, -0.16047446, -0.25127971, ...,  0.0887628 ,
       -0.12791109,  0.0545721 ])

In [ ]:
# Create a sim_dict, similarity dictionary based on the BERT embeddings.
sim_dict = {}
for key1,val1 in dict_emb.items():
  sim_dict[key1] = {}
  for key2,val2 in dict_emb.items():
    sim_dict[key1][key2] = 1 - spatial.distance.cosine(val1, val2)

In [ ]:
sim_dict["3.txt"]["5.txt"]

0.9879263161124842

In [ ]:
# Create a 100 x 100 similarity matrix based on the BERT embeddings. 
sim_mat = np.zeros((100,100)) # Forming 2D array using numpy
for i in range(100):
  for j in range(100):
    sim_mat[i][j] = 1 - spatial.distance.cosine(dict_emb[str(i + 1) + ".txt"], dict_emb[str(j + 1) + ".txt"])

In [ ]:
sim_mat[2][4]

0.9879263161124842

In [ ]:
sim_mat

array([[1.        , 0.98117578, 0.98731814, ..., 0.97945429, 0.98367163,
        0.97018314],
       [0.98117578, 1.        , 0.98934483, ..., 0.9654464 , 0.97001454,
        0.95931779],
       [0.98731814, 0.98934483, 1.        , ..., 0.97414345, 0.98099378,
        0.96821931],
       ...,
       [0.97945429, 0.9654464 , 0.97414345, ..., 1.        , 0.98483431,
        0.96966173],
       [0.98367163, 0.97001454, 0.98099378, ..., 0.98483431, 1.        ,
        0.9741753 ],
       [0.97018314, 0.95931779, 0.96821931, ..., 0.96966173, 0.9741753 ,
        1.        ]])

In [ ]:
# Finding the minimum similarity value between articles.
# This is quite high for a minimum.
np.min(sim_mat)

0.9243145520623182

<h2>Using Universal Sentence Encoder instead of BERT</h2>

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/2" 
embed = hub.Module(module_url)

In [ ]:
# Could not do sentence embeddings for each sentence in the document -- used up too much RAM and crashed the session.
dict_USE_emb = {}
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  for key, value in dict_art.items():
    sentences = nltk.tokenize.sent_tokenize(value)
    embeddings = np.array(session.run(embed(sentences)))
    # using mean sentence embeddings to calc 'document embedding'
    dict_USE_emb[key] = np.average(embeddings, axis = 0) # Returns an average array

In [ ]:
# Instead we try document embedding using USE because USE also supports short paragraphs
# There is no hard limit on how long the paragraph is. Roughly, the longer it is, the more 'diluted' the embedding will be.

dict_emb_USE = {}
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  embeddings = session.run(embed(list(dict_art.values())))
  keys = list(dict_art.keys())
  for i in range(100):
    dict_emb_USE[keys[i]] = embeddings[i]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
dict_emb_USE

{'1.txt': array([-0.00084906, -0.05520914,  0.06778991, -0.00551787, -0.00259336,
        -0.03128554,  0.01911337, -0.01595762, -0.04820832,  0.06239307,
        -0.00205393, -0.07125825, -0.01932998,  0.07805299, -0.00996943,
         0.02126331,  0.00456544, -0.00677511,  0.02571699,  0.08132052,
        -0.05752527, -0.02266333, -0.00047547, -0.04022417, -0.04693603,
         0.04254109,  0.04523459, -0.02129346, -0.02594743, -0.0115415 ,
        -0.02986079, -0.0258997 ,  0.04852584, -0.01122223, -0.01935449,
         0.01084227,  0.01846333,  0.01443861,  0.00473363,  0.03930781,
         0.0087689 ,  0.0063544 ,  0.00758575,  0.05094009,  0.03243709,
        -0.00227676,  0.03339943, -0.01375325,  0.01931048, -0.01509143,
        -0.00139631,  0.05011079, -0.02909961, -0.10643648, -0.00027897,
        -0.04174632,  0.01093074, -0.08333954, -0.02037905,  0.0208442 ,
         0.00111435,  0.00748043,  0.0636775 ,  0.00193958,  0.04907992,
        -0.00139891, -0.02311454,  0.03955

In [ ]:
# Create a sim_dict, similarity dictionary based on the USE embeddings.
sim_dict_USE = {}
for key1,val1 in dict_emb_USE.items():
  sim_dict_USE[key1] = {}
  for key2,val2 in dict_emb_USE.items():
    sim_dict_USE[key1][key2] = 1 - spatial.distance.cosine(val1, val2)

In [ ]:
sim_dict_USE["3.txt"]["5.txt"]

0.7174440622329712

In [ ]:
# Create a 100 x 100 similarity matrix based on the BERT embeddings. 
sim_mat_USE = np.zeros((100,100)) # Forming 2D array using numpy
for i in range(100):
  for j in range(100):
    sim_mat_USE[i][j] = 1 - spatial.distance.cosine(dict_emb_USE[str(i + 1) + ".txt"], dict_emb_USE[str(j + 1) + ".txt"])

In [ ]:
sim_mat_USE

array([[1.        , 0.70237154, 0.63063842, ..., 0.44120258, 0.20868428,
        0.32984573],
       [0.70237154, 1.        , 0.77705938, ..., 0.39825034, 0.12543635,
        0.29985949],
       [0.63063842, 0.77705938, 1.        , ..., 0.37838346, 0.24470595,
        0.25365454],
       ...,
       [0.44120258, 0.39825034, 0.37838346, ..., 1.        , 0.31903598,
        0.37326175],
       [0.20868428, 0.12543635, 0.24470595, ..., 0.31903598, 1.        ,
        0.21586895],
       [0.32984573, 0.29985949, 0.25365454, ..., 0.37326175, 0.21586895,
        1.        ]])

In [ ]:
np.min(sim_mat_USE)

0.054705046117305756

In [ ]:
# Max value excluding diagonals
mask = np.ones(sim_mat_USE.shape, dtype=bool)
np.fill_diagonal(mask, 0)
sim_mat_USE[mask].max()

0.9516576528549194

<h2>Comparing USE and BERT</h2>

<h4>Irrelevant Sentences</h4>

In [ ]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), 
                 tf.tables_initializer()])
    test_USE = session.run(embed(["How old are you?", "Have some pineapples."]))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
1 - spatial.distance.cosine(test_USE[0], test_USE[1])

0.4304225444793701

In [ ]:
test_bert = get_embeddings(["How old are you?", "Have some pineapples."])

In [ ]:
1 - spatial.distance.cosine(test_bert[0], test_bert[1])

0.8412800285875349

<h4>Sentences with similar contexts/meanings</h4>

In [ ]:
test_bert2 = get_embeddings(["You look amazing.", "You are not ugly."])

In [ ]:
1 - spatial.distance.cosine(test_bert2[0], test_bert2[1])

0.8885950580727265

In [ ]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), 
                 tf.tables_initializer()])
    test_USE2 = session.run(embed(["You look amazing.", "You are not ugly."]))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
1 - spatial.distance.cosine(test_USE2[0], test_USE2[1])

0.7821219563484192

In [ ]:
test_bert3 = get_embeddings(["He's heavier than I am.", "He weighs more than I do."])

In [ ]:
1 - spatial.distance.cosine(test_bert3[0], test_bert3[1])

0.9734720464529293

In [ ]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), 
                 tf.tables_initializer()])
    test_USE3 = session.run(embed(["He's heavier than I am.", "He weighs more than I do."]))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
1 - spatial.distance.cosine(test_USE3[0], test_USE3[1])

0.912887692451477

<h2>Previous Draft Codes</h2>

In [ ]:
!bert-serving-start -model_dir /content/drive/My\ Drive/DSTA\ Internship/uncased_L-24_H-1024_A-16/uncased_L-24_H-1024_A-16/

In [ ]:
text_embeddings = get_embeddings(["Hello, my name is Shirley.","Hello, we are here!"])

In [ ]:
text_embeddings

[[-0.46242767572402954,
  0.14673691987991333,
  -0.17444784939289093,
  -0.15033622086048126,
  -0.14476323127746582,
  -0.5319898128509521,
  0.21811281144618988,
  0.6331086754798889,
  0.13632169365882874,
  0.41882437467575073,
  -0.10239659994840622,
  -0.2776552438735962,
  -0.26848524808883667,
  -0.15075252950191498,
  -0.4309755861759186,
  -0.4862912595272064,
  -0.39249667525291443,
  -0.4078425467014313,
  0.23615707457065582,
  -0.36301374435424805,
  0.3463802933692932,
  0.04049450531601906,
  -0.24250449240207672,
  -0.05674280226230621,
  -0.6244709491729736,
  0.48859331011772156,
  0.9079447984695435,
  -0.41560813784599304,
  -0.17976945638656616,
  0.1229940801858902,
  -0.23528200387954712,
  0.5511993765830994,
  0.04205768182873726,
  0.07087069004774094,
  -0.3229125738143921,
  -0.06067393720149994,
  0.7002816796302795,
  0.05190083011984825,
  -0.1742599457502365,
  0.175090491771698,
  0.39012351632118225,
  0.5754984617233276,
  0.005426315125077963,
  -0

In [ ]:
similarity = 1 - spatial.distance.cosine(text_embeddings[0], text_embeddings[1])
similarity